In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["MLFLOW_TRACKING_URI"] = os.getenv("MLFLOW_TRACKING_URI")
os.environ["MLFLOW_TRACKING_USERNAME"]=os.getenv("MLFLOW_TRACKING_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"]=os.getenv("MLFLOW_TRACKING_PASSWORD")


In [2]:
import os
os.chdir("../")
%pwd

'/Users/hh/MLops/dataScienceProject'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    experiment_name: str

In [4]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json
load_dotenv()

True

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
        root_dir=config.root_dir,
        test_data_path=config.test_data_path,
        model_path=config.model_path,
        all_params=params,
        metric_file_name=config.metric_file_name,
        target_column=schema.name,
        mlflow_uri= os.getenv("MLFLOW_TRACKING_URI"),
        experiment_name = config.experiment_name
        )
        
        return model_evaluation_config


In [28]:
import os
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib
from src.datascience import logger
from mlflow.models import infer_signature

In [35]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def evaluate_metrics(self, actual, predicted):
        rsme = np.sqrt(mean_squared_error(actual,predicted))
        mae = mean_absolute_error(actual,predicted)
        r2 = r2_score(actual, predicted)
        return rsme, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_experiment(self.config.experiment_name)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rsme, mae, r2) = self.evaluate_metrics(test_y, predicted_qualities)

            scores = {"rsme":rsme, "mae":mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rsme", rsme)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            mlflow.log_artifact(self.config.model_path, artifact_path="model")


In [36]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    logger.exception(e)
    raise e

[2025-08-11 13:00:33,082: INFO: common: yaml file config/config.yaml is loaded successfully]
[2025-08-11 13:00:33,083: INFO: common: yaml file params.yaml is loaded successfully]
[2025-08-11 13:00:33,088: INFO: common: yaml file schema.yaml is loaded successfully]
[2025-08-11 13:00:33,089: INFO: common: created directory at artifacts]
[2025-08-11 13:00:33,090: INFO: common: created directory at artifacts/model_evaluation]
[2025-08-11 13:00:34,490: INFO: common: json file saved at artifacts/model_evaluation/metrics.json]
🏃 View run stately-moose-827 at: https://dagshub.com/prc1s/dataScienceProject.mlflow/#/experiments/0/runs/d060484b48234a8fb881dfe2ee8b1dbd
🧪 View experiment at: https://dagshub.com/prc1s/dataScienceProject.mlflow/#/experiments/0
